# Recursos Especiais interpostos pelo MPSP
### 17/06/2022

O presente notebook buscou obter os resultados dos recursos especiais interpostos pelo MPSP.

A análise foi feita a partir de dataset gerado pelo STJ, filtrando-se 1000 processos que tinham o MPSP como parte ou interessado, em ordem decrescente de autuação. O arquivo "csv" foi gerado pelo STJ no dia 17 jun. 2022.

O codigo foi escrito em Python. Foram utilizadas as bibliotecas pandas (para as operações com o dataframe), matplotlib (para os gráficos), requests (para as requisições http ao site do STJ) e beautifulsoup (para a extração dos dados dos documentos html recuperados).


In [1]:
# Importação das bibliotecas necessárias

import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Criação do dataframe

df = pd.read_csv('processosSTJ_resp_todos_data_de_autuacao_1000.csv', parse_dates=['Data Autuação'])
df = df.dropna(subset='Processo') # Exclusão dos registros sem o nº de processo.

In [3]:
print(f'Total de processos encontrados: {len(df)}')

Total de processos encontrados: 965


In [4]:
intervalo_autuacao = f"{str(df['Data Autuação'].min())[0:10]} a {str(df['Data Autuação'].max())[0:10]}"
print(f"Intervado das datas de autuação: {intervalo_autuacao}")
print(f"Total: {str(df['Data Autuação'].max() - df['Data Autuação'].min())[0:7]}")

Intervado das datas de autuação: 2022-01-09 a 2022-06-17
Total: 159 day


In [5]:
df.sample(5)

,Número de Registro,Número Único,Processo,Nome Parte Autora,Nome Parte Ré,Data da Última Fase,Fase,Tramitando,Orgão Julgador,Segredo de Justiça,Ministro Relator,Observações,Data Autuação
694,202200473548,15007492420198260180,REsp 1986673,ACACIO ORMASTRONI,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,2022-05-11T23:30:45.000+0000,Conclusos para decisão ao(à) Ministro(a) REYNA...,S,QUINTA TURMA,N,REYNALDO SOARES DA FONSECA,NaN,2022-02-24
641,202200569041,00006566320188260571,REsp 1988016,DANIEL BOTELHO HORN,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,2022-06-08T21:45:39.000+0000,Conclusos para decisão ao(à) Ministro(a) JESUÍ...,S,QUINTA TURMA,N,JESUÍNO RISSATO (DESEMBARGADOR CONVOCADO DO TJ...,NaN,2022-03-04
865,202200284402,00024323720178260344,REsp 1983525,LUCAS EMANUEL BONFIM FERNANDES,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,2022-04-20T06:35:46.000+0000,Baixa Definitiva para TRIBUNAL DE JUSTIÇA DO E...,N,SEXTA TURMA,N,ANTONIO SALDANHA PALHEIRO,NaN,2022-02-08
418,202200970998,15021805820208260536,REsp 1995266,CAIO SAMPAIO SANTOS,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,2022-05-18T22:16:46.000+0000,Conclusos para decisão ao(à) Ministro(a) RIBEI...,S,QUINTA TURMA,N,RIBEIRO DANTAS,NaN,2022-04-07
168,202201673381,15198344620198260228,REsp 2005698,JOAO VITOR DA SILVA ALMEIDA,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,2022-06-15T13:01:23.000+0000,DEFENSORIA PÚBLICA DO ESTADO DE SÃO PAULO inti...,S,QUINTA TURMA,N,REYNALDO SOARES DA FONSECA,NaN,2022-06-01


In [6]:
df['Nome Parte Autora'].value_counts()

MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO    164
LUIZ CARLOS DA SILVA                           2
CAIQUE FRANCISCO DA SILVA                      2
STENIO AUGUSTO VASQUES BALDIN                  2
JOSE CLAUDIO SOARES                            1
                                            ... 
DAVID PEREIRA CASELLA                          1
CRISTIANO SANTOS FABRICIO                      1
JOAO GUILHERME SANTOS TEIXEIRA                 1
MARCOS HENRIQUE ROCHA MASSARI                  1
B A G                                          1
Name: Nome Parte Autora, Length: 798, dtype: int64

### Restringindo o DF aos casos de REsp em que o MPSP é o autor

In [7]:
criterio = df['Nome Parte Autora'] == 'MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO'
df = df[criterio]
print (len(df))

164


In [8]:
# Preparação do dataframe para o webscraping

df['Número de Registro'] = df['Número de Registro'].astype(str)
df['Monocrática'] = ''
df['Acórdão'] = ''

# Funções 

def requisita_monocratica(registro):
    """Retorna o texto da decisão monocrática em para o registro fornecido"""
    response = requests.post(f'https://scon.stj.jus.br/SCON/pesquisar.jsp?livre={registro}&b=DTXT&p=true&tp=T')
    html = response.text # ver: https://stackoverflow.com/questions/17011357/what-is-the-difference-between-content-and-text 
    return html

def requisita_acordao(registro):
    """Retorna o texto do acórdao ou ementa para o registro fornecido"""
    response = requests.post(f'https://scon.stj.jus.br/SCON/pesquisar.jsp?livre={registro}&b=ACOR&p=true&tp=T')
    html = response.text # ver: https://stackoverflow.com/questions/17011357/what-is-the-difference-between-content-and-text 
    return html

def encontra_conteudo (html):
    """Retorna o conteúdo da id em formato texto e sem as tags do html"""
    soup = BeautifulSoup(html, 'html.parser')
    conteudo = soup.find(id="Semformatacao1") # com as tags
    return conteudo.text # sem as tags

In [9]:
# Percorrendo o dataframe

for i in df.index:
    registro = df.loc[i, 'Número de Registro']
    html = requisita_monocratica(registro)
    try:
        df.loc[i, 'Monocrática'] = encontra_conteudo(html)
    except:
        df.loc[i, 'Monocrática'] = 'Não encontrado'
    html = requisita_acordao(registro)
    try:
        df.loc[i, 'Acórdão'] = encontra_conteudo(html)
    except:
        df.loc[i, 'Acórdão'] = 'Não encontrado'

In [10]:
df.sample(5)

,Número de Registro,Número Único,Processo,Nome Parte Autora,Nome Parte Ré,Data da Última Fase,Fase,Tramitando,Orgão Julgador,Segredo de Justiça,Ministro Relator,Observações,Data Autuação,Monocrática,Acórdão
605,202200663834,00113481420148260361,REsp 1989509,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,JEFERSON JOSE DE SANTANA,2022-04-28T01:05:28.000+0000,Baixa Definitiva para TRIBUNAL DE JUSTIÇA DO E...,N,QUINTA TURMA,N,JESUÍNO RISSATO (DESEMBARGADOR CONVOCADO DO TJ...,NaN,2022-03-11,\n\r\nRECURSO ESPECIAL Nº 1989509 - SP (2022/0...,Não encontrado
426,202200981184,00015501620158260548,REsp 1995312,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,GIOVANI JARDULLI,2022-04-18T23:46:17.000+0000,Conclusos para decisão ao(à) Ministro(a) SEBAS...,S,SEXTA TURMA,N,SEBASTIÃO REIS JÚNIOR,NaN,2022-04-07,Não encontrado,Não encontrado
512,202200793720,00877725520178260050,REsp 1991918,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,WESLLEY ISMAEL BARBOZA,2022-04-06T17:31:06.000+0000,Conclusos para decisão ao(à) Ministro(a) RIBEI...,S,QUINTA TURMA,N,RIBEIRO DANTAS,NaN,2022-03-22,Não encontrado,Não encontrado
66,202201766498,22240452120218260000,REsp 2006775,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,RENAN ANDRE DOS SANTOS,2022-06-15T18:16:09.000+0000,Disponibilizada cópia digital dos autos à(o) M...,S,QUINTA TURMA,N,JOEL ILAN PACIORNIK,NaN,2022-06-09,Não encontrado,Não encontrado
303,202201236180,15003148620208260286,REsp 1999431,MINISTÉRIO PÚBLICO DO ESTADO DE SÃO PAULO,GLEISON MARINHO DIAS,2022-05-17T10:31:06.000+0000,Conclusos para decisão ao(à) Ministro(a) REYNA...,S,QUINTA TURMA,N,REYNALDO SOARES DA FONSECA,NaN,2022-05-03,Não encontrado,Não encontrado


In [11]:
# Exportando o resultado para planilha Excel

df.to_excel('REsp.xlsx', engine='xlsxwriter')

In [12]:
# REsp com decisões monocráticas ou acórdãos

criterio = (df['Monocrática'] != 'Não encontrado') | (df['Acórdão'] != 'Não encontrado')
df1 = df[criterio]
print(f"Total de REsp com decisões monocráticas ou acórdãos: {len(df1)}")

Total de REsp com decisões monocráticas ou acórdãos: 59


In [13]:
# Exportando o resultado para arquivo texto

with open ('REsp.txt', 'w') as f:
    f.write(f"RECURSOS ESPECIAIS ENTRE {intervalo_autuacao}\n\n")
    f.write(f"- Nº de REsp pesquisados: {len(df)}\n\n")
    f.write(f"Total de REsp com decisões monocráticas ou acórdãos: {len(df1)}\n\n")
    f.write("__________\n\n")
    
    for i in df1.index:
        f.write(f"Data Autuação: {df1['Data Autuação'][i]} - Processo: {df1['Processo'][i]}\n")
        f.write(f"Ministro Relator: {df1['Ministro Relator'][i]} - Órgão Julgador: {df1['Orgão Julgador'][i]}\n")
        f.write(f"Nome Parte Autora: {df1['Nome Parte Autora'][i]}\n")
        f.write(f"Nome Parte Ré: {df1['Nome Parte Ré'][i]}\n")
        f.write(f"Data da Última Fase: {str(df1['Data da Última Fase'][i])[0:10]}\n")
        f.write("__________\n\n")
        f.write(f"DECISÃO MONOCRÁTICA:\n{df1['Monocrática'][i]}\n\n")
        f.write(f"ACÓRDÃO:\n{df1['Acórdão'][i]}\n")
        f.write(f"\n**************** Fim dos dados do {df1['Processo'][i]} ****************\n\n")